In [ ]:
# Dependencias
!pip install pypdf langchain langchain-community langchain-core langchain-openai chromadb python-dotenv

# Usar solo en Databricks
# dbutils.library.restartPython()

# Ejercicio de Ingeniería de Software con Inteligencia Artificial

## Descripción de la Prueba

El objetivo de esta prueba es evaluar tus habilidades y conocimientos en el desarrollo de software con inteligencia artificial, específicamente en sistemas RAG (Retrieval-Augmented Generation), embeddings, y bases de datos vectoriales.

Se te proporcionarán tres archivos PDF con reportes de resultados de Grupo Bimbo, así como las llaves de API y los endpoints de OpenAI. Tu tarea será utilizar estos recursos para desarrollar un sistema que pueda realizar tareas de retrieval de manera eficiente.

## Objetivos de la Prueba

1. Comprensión de los datos: Deberás ser capaz de procesar y entender los datos proporcionados en los archivos PDF.

2. Uso de OpenAI y embeddings: Deberás demostrar tu habilidad para trabajar con OpenAI y embeddings para realizar tareas de retrieval.

3. Implementación de bases de datos vectoriales: Deberás implementar una base de datos vectorial para manejar eficientemente los datos.

4. Desarrollo de un sistema RAG: Finalmente, deberás demostrar tu capacidad para desarrollar un sistema RAG que pueda realizar tareas de retrieval y generación de texto de manera eficiente.

## Presentación del Código

Tu código debe seguir las mejores prácticas de codificación y puede presentarse en notebooks de Jupyter. Esto incluye:

· Legibilidad: El código debe ser fácil de leer y entender.

· Comentarios: Debes incluir comentarios que expliquen tu razonamiento y las decisiones de diseño que tomaste.

· Estructura: El código debe estar bien estructurado y organizado.

· Pruebas: Debes incluir pruebas para asegurar que tu código funciona como se espera.

Para revisión de este proyecto ser realizará una sesión en línea donde presentarás tu abordaje el problema, tu resolución técnica y código generado.

- APIkey: 75433f0e2ce040ebb90a2fa457fbc815
- EndPoint: https://ennovenoai.openai.azure.com/
- Modelo Openai: EnnovenGPTTurbo
- Modelo Ada-002: EnnovenAda

Anexo igualmente los PDFs que sirven como fuente de datos para el sistema RAG, recordando que buscamos una búsqueda enriquecida: (Datos + metadata)

El archivo contiene tablas, e imágenes, así como textos. Se espera del ejercicio el procesamiento de las tablas e imágenes dentro del modelo. Así como una aproximación de reranking de los resultados para un mejor contexto de respuesta.

Las API Keys serán cambiadas una ves terminado el ejercicio.

In [51]:
# Bibliotecas para interactuar con el LLM
from langchain_openai import AzureChatOpenAI
# Para generar los embeddings
from langchain_openai import AzureOpenAIEmbeddings

# Para cargar los datos en formato PDF
from langchain_community.document_loaders import PyPDFLoader
# Para el splitting de los textos
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Para almacenar los embeddings en ChromaDB
from langchain_community.vectorstores import Chroma
# Para generar el RAG usando la base de datos vectorial
from langchain.chains import RetrievalQAWithSourcesChain
# Para crear un template y obtener respuestas personalizadas
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# Interactuar con el sistema de archivos
import os
import glob
from dotenv import load_dotenv

# Cargar las variables de ambiente desde el .env
load_dotenv()

True

## Credenciales de accesso

Para Databricks: Las credenciales de acceso al modelo han sido almacenadas en variables de ambiente en la configuración del Clúster para mayor seguridad.

<img src="./img/databricks-env-variables.png">

Para entornos de prueba local: Las credenciales se encuentran en un archivo `.env`

<img src="./img/variables-de-ambiente.png">

In [2]:
openai_api_version = os.environ["OPENAI_API_VERSION"]
azure_deployment = os.environ["AZURE_DEPLOYMENT"]
azure_openai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
model_name = os.environ["MODEL_NAME"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

Creamos una instancia para usar el modelo de ChatGPT de Azure y el modelo para generar los embeddings

In [3]:
# Crear una conexión con el modelo GPT de Azure
model = AzureChatOpenAI(
    api_key = azure_openai_api_key, # API Key
    azure_endpoint = azure_openai_endpoint, # Endpoint
    openai_api_version = openai_api_version, # Obtenido de la documentación
    model_name = model_name, # Nombre del modelo
    temperature = 0 # Temperatura del modelo GPT
)

In [4]:
# Definir el modelo para generar los embeddings
embedding_model = AzureOpenAIEmbeddings(
    api_key = azure_openai_api_key, # API Key
    azure_endpoint = azure_openai_endpoint, # Endpoint
    azure_deployment = azure_deployment, # Nombre del modelo de embeddings
    openai_api_version = openai_api_version # Obtenido de la documentación
)

In [39]:
def get_splitter(chunk_size : int, chunk_overlap : int, separators : list | None = None) -> RecursiveCharacterTextSplitter:
    """
    Función que permite instanciar un objeto para splitting de caracteres recursivamente
    """
    # Inicializar el splitter con los hiperparametros
    splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size, 
        chunk_overlap = chunk_overlap,
        separators = separators
    )

    return splitter

In [40]:
def get_database(directory : str, model = AzureOpenAIEmbeddings, collection = str) -> Chroma:
    """
    Función que permite crear una base de datos vectorial usando ChromaDB
    """

    # Inicializar la base de datos vectorial
    vectordb = Chroma(
        persist_directory = directory,
        embedding_function = model,
        collection_name = collection
    )

    # Configurar Chroma para hacer datos persistentes en disco
    vectordb.persist()
    
    return vectordb

In [41]:
# Instanciar el splitter de texto
splitter = get_splitter(chunk_size = 1000, chunk_overlap = 200, separators = ["."])
splitter

In [42]:
# Instanciar una conexión con la base de datos vectorial
vectordb = get_database(directory = "db", model = embedding_model, collection = "ennoven_db")
vectordb

A continuación, se muestra un ciclo de trabajo en el que se leen los archivos PDF que se encuentran en el directorio `data` y se procesan uno por uno.

Las tres etapas se definen a continuación:

## Etapa 1: Ingesta de datos (usando Loaders)

Dependiendo del caso de uso, pueden existir multiples tipos de formatos de archivos que quisiera llegar a procesarse.

LangChain ofrece una extensa variedad de Loaders que permiten cargar estos datos que pueden consultarse aquí: https://python.langchain.com/docs/integrations/document_loaders

En este caso, estamos usando un loader especial para cargar archivos PDF.

## Etapa 2: Separación de los textos por lotes (chunks)

Separar los textos permite obtener porciones fáciles de procesar y utilizar por el modelo de IA para ofrecer un contexto por similitud. Este proceso involucra establecer una logitud de caracteres y una unión de información entre estos (overlap).

Los parametros que permiten realizar esto son `chunk_size` y `chunk_overlap` respectivamente. 

Para la separación de los textos, se está usando `RecursiveCharacterTextSplitter`.

## Etapa 3: Crear representaciones vectoriales de los chunks 

Las representaciones vectoriales permite convertir texto en vectores numericos que permiten mantener el contexto, semantica y similitud entre los chunks de textos que separamos anteriormente. 

Para generar los embeddings, tenemos que usar un modelo especialmente para esto, en este caso, estamos usando `EnnovenAda`, aunque existen muchos otros que pueden encontrarse tanto en OpenAI (embeddings propietario) como en TheHuggingFace

<img src="./img/embedding-models.png" width = "1200px">

In [43]:
# Para cada PDF dentro del directorio
# for file in os.listdir("data"):
for file in glob.glob("data/*.pdf"):
# Crear una conexión con el PDF
    print(f"Processing file: {file}")
    loader = PyPDFLoader(f"{file}")
    
    # Paso 1: Extraer el texto del PDF
    print(f"Extracting text from file: {file}")
    pdf_data = loader.load()
    
    # Paso 2: Dividir el contenido del PDF en documentos
    print(f"Splitting docs for {file}")
    docs = splitter.split_documents(pdf_data)
    
    # Paso 3: Crear los embeddings de los documentos usando el modelo y almacenarlos en ChromaDB
    print(f"Creating embeddings for docs in {file}")
    docstorage = Chroma.from_documents(docs, embedding_model)

    print("Embeddings created", end="\n\n")

Processing file: data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf
Extracting text from file: data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf
Splitting docs for data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf
Creating embeddings for docs in data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf
Embeddings created

Processing file: data/Reporte_Definitivo_BMV_XBRL_Español_Jun_23.pdf
Extracting text from file: data/Reporte_Definitivo_BMV_XBRL_Español_Jun_23.pdf
Splitting docs for data/Reporte_Definitivo_BMV_XBRL_Español_Jun_23.pdf
Creating embeddings for docs in data/Reporte_Definitivo_BMV_XBRL_Español_Jun_23.pdf
Embeddings created

Processing file: data/Reporte_Definitivo_BMV_XBRL_Español_Sep_23.pdf
Extracting text from file: data/Reporte_Definitivo_BMV_XBRL_Español_Sep_23.pdf
Splitting docs for data/Reporte_Definitivo_BMV_XBRL_Español_Sep_23.pdf
Creating embeddings for docs in data/Reporte_Definitivo_BMV_XBRL_Español_Sep_23.pdf
Embeddings created



## Realizar consultas sin formato

Una vez almacenados los embeddings en la base de datos vectorias, podemos usarla para construir un RAG (básico sin formato de salida) para obtener en base al contexto de los archivos PDF que extraímos.

Usamos la función `RetrievalQAWithSourcesChain` para obtener una respuesta junto con su metadata, que incluye la fuente de donde estos datos provinieron. Especialmente útil para identificar si la respuesta generada por el modelo ha sido una alucinación.

In [45]:
# Retornar las referencias al RAG desde la base de datos vectorial de ChormaDB
# para generar una interfaz de comunicacion entre el modelo de IA y la base de datos vectorial para su consumo
print(f"Returning references to RAG data sources")
rag = RetrievalQAWithSourcesChain.from_chain_type( 
    llm = model, # Usar el modelo GPT
    chain_type = "stuff",
    retriever = docstorage.as_retriever() # Usar el contenido de la base de datos
)

Returning references to RAG data sources


Para probar este modelo primitivo, podemos usar 

In [46]:
query = "¿Cuáles son las instituciones bancarias en las que Grupo Bimbo tiene un endeudamiento?"
rag.invoke(query)

{'question': '¿Cuáles son las instituciones bancarias en las que Grupo Bimbo tiene un endeudamiento?',
 'answer': 'Grupo Bimbo tiene un endeudamiento con las siguientes instituciones bancarias: BBVA Bancomer S.A., Bank of America N.A., Citibank N.A., Coöperatieve Rabobank U.A., New York HSBC México S.A., ING Bank N.V., JP Morgan Chase Bank N.A., Mizuho Bank, Ltd, Morgan Stanley Bank, N.A., MUFG Bank, Ltd. y Banco Santander S.A.\n',
 'sources': 'data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf, data/Reporte_Definitivo_BMV_XBRL_Español_Sep_23.pdf'}

In [47]:
query = "¿Cuáles son los puntos principales de los papers?"
rag.invoke(query)

{'question': '¿Cuáles son los puntos principales de los papers?',
 'answer': 'Los puntos principales de los papers son:\n1. La finalidad primordial es lograr una posición neutral y equilibrada con relación a la exposición al riesgo de una cierta variable financiera.\n2. Las estrategias de cobertura son valuadas y monitoreadas de manera formal y continua.\n3. Las operaciones con instrumentos financieros derivados relacionados a materias primas son principalmente celebradas en los mercados reconocidos como el Minneapolis Grain Exchange (MGE), Kansas City Board of Trade (KCBOT), Chicago Board of Trade (CBOT), New York Mercantile Exchange (NYMEX) y Mercado de Término de Buenos Aires (MATba).\n4. También se han realizado operaciones bilaterales ligadas a la cobertura de materias primas.\n',
 'sources': 'data/Reporte_Definitivo_BMV_XBRL_Español_Jun_23.pdf, data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf, data/Reporte_Definitivo_BMV_XBRL_Español_Sep_23.pdf'}

In [48]:
query = "¿Cuáles son las instituciones bancarias en las que Grupo Bimbo tiene un endeudamiento?"
matching = docstorage.similarity_search(query)
answer = rag.invoke(query)
print("EnnovenGPTTurbo: ", answer["answer"], end = "\n\n")
print("Sources: ", answer["sources"], end = "\n\n")
print("ChromaDB similarity:\n", matching[0].page_content, end = "\n\n")

EnnovenGPTTurbo:  Grupo Bimbo tiene un endeudamiento con las siguientes instituciones bancarias: BBVA Bancomer S.A., Bank of America N.A., Citibank N.A., Coöperatieve Rabobank U.A., New York HSBC México S.A., ING Bank N.V., JP Morgan Chase Bank N.A., Mizuho Bank, Ltd, Morgan Stanley Bank, N.A., MUFG Bank, Ltd. y Banco Santander S.A.


Sources:  data/Reporte_Definitivo_BMV_XBRL_Español_Mar_23.pdf, data/Reporte_Definitivo_BMV_XBRL_Español_Sep_23.pdf

ChromaDB similarity:
 BIMBO Consolidado
Clave de Cotización:       BIMBO Trimestre:      1     Año:    2023
69 de 172El 15 de marzo de 2023, la Compañía 
renovó su línea de crédito revolvente 
comprometida, sindicada y multimoneda, 
la cual está vinculada a la 
sustentabilidad. Las instituciones 
financieras que participan en esta 
línea son BBVA Bancomer S.A., Bank of 
America N.A., Citibank N.A., 
Coöperatieve Rabobank U.A., New York 
HSBC México S.A., ING Bank N.V., JP 
Morgan Chase Bank N.A., Mizuho Bank, 
Ltd, Morgan Stanley Bank, N.A

## Utilizar la AI y la base de datos vectorial para realizar consultas

Para realizar las consultas, se pueden configurar los prompts para que aporten el contexto directo de la base de datos vectorial, así como una plantilla de la respuesta esperada para el usuario.

Para ello, tambien estamos usando LCEL, que es una forma de encadenamiento de ordenes usando el pipe operator.

In [53]:
# Creamos un template para la IA
template = """Responde la siguiente pregunta usando el contexto: {context}. Pregunta: {question}"""

# Creamos el prompt 
prompt = ChatPromptTemplate.from_template(template)
# Creamos una cadena y la ejecutamos
chain = (
    {"context" : docstorage.as_retriever(), "question" : RunnablePassthrough()} 
    | prompt | model
)



In [55]:
chain.invoke("¿Cuáles son las instituciones bancarias en las que Grupo Bimbo tiene un endeudamiento?")

AIMessage(content='Las instituciones bancarias en las que Grupo Bimbo tiene un endeudamiento son BBVA Bancomer S.A., Bank of America N.A., Citibank N.A., Coöperatieve Rabobank U.A., New York HSBC México S.A., ING Bank N.V., JP Morgan Chase Bank N.A., Mizuho Bank, Ltd, Morgan Stanley Bank, N.A., MUFG Bank, Ltd. y Banco Santander S.A.')